<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff603b;'>Modules, Helpful Functions, & Styling</h1>

In [ ]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Ewert|Aladin&effect=3d');
span {color:black; text-shadow:3px 3px 3px #aaa;}  
div.output_area pre{font-family:Aladin; font-size:110%; color:#ff603b;}      
</style>

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import h5py,urllib,pylab as pl,seaborn as sn
import numpy as np,pandas as pd
import tensorflow_hub as th,tensorflow as tf
import tensorflow.keras.layers as tkl
from IPython.display import display,HTML
import PIL.Image as pimg
data_path='https://raw.githubusercontent.com/'+\
          'OlgaBelitskaya/data_kitchen/main/'
url_path='https://olgabelitskaya.gitlab.io/images/'
file_path='../input/image-examples-for-mixed-styles/'
file_path2='../input/flower-color-images/'
hub_path='https://tfhub.dev/google/magenta/'+\
         'arbitrary-image-stylization-v1-256/1'
hub_path2='https://tfhub.dev/google/faster_rcnn/'+\
          'openimages_v4/inception_resnet_v2/1'
model_weights='/tmp/checkpoint'
col='#ff603b'

In [ ]:
def get_file(file,url_path=url_path):
    input_file=urllib.request.urlopen(url_path+file)
    output_file=open(file,'wb'); 
    output_file.write(input_file.read())
    output_file.close(); input_file.close()
def prepro(x_train,y_train,x_test,y_test):
    n=int(len(x_test)/2)
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    df=pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],
                     [y_train.shape,y_valid.shape,y_test.shape]],
                    columns=['train','valid','test'],
                    index=['images','labels'])
    display(df)
    return [[x_train,x_valid,x_test],
            [y_train,y_valid,y_test]]
def load_img(path2img,max_dim=512):
    img=tf.io.read_file(path2img)
    img=tf.image.decode_image(img,channels=3)
    img=tf.image.convert_image_dtype(img,tf.float32)
    shape=tf.cast(tf.shape(img)[:-1],tf.float32)
    scale=max_dim/max(shape)
    new_shape=tf.cast(shape*scale,tf.int32)
    img=tf.image.resize(img,new_shape)
    return img[tf.newaxis,:]
def tensor2img(tensor):
    tensor=np.array(tensor*255,dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0]==1
        tensor=tensor[0]
    return pimg.fromarray(tensor)

<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff603b;'>Data Loading & Preprocessing</h1>

In [ ]:
(x_train1,y_train1),(x_test1,y_test1)=\
tf.keras.datasets.cifar10.load_data()
[[x_train1,x_valid1,x_test1],
 [y_train1,y_valid1,y_test1]]=\
prepro(x_train1,y_train1,x_test1,y_test1)

In [ ]:
cifar_classes=['airplane','automobile','bird','cat','deer',
              'dog','frog','horse','ship','truck']
num_classes1=len(cifar_classes); img_size1=x_train1.shape[1]
n=np.random.randint(0,x_train1.shape[0])
pl.figure(figsize=(2,2)); pl.xticks([]); pl.yticks([])
pl.title(cifar_classes[y_train1[n][0]],fontsize=16,color=col)
pl.imshow(x_train1[n]); pl.tight_layout();

In [ ]:
data_file='Flowers128.h5'
get_file(data_file,data_path)
with h5py.File(data_file,'r') as f:
    keys=list(f.keys())
    print('h5py.File keys: '+', '.join(keys))
    images=np.array(f[keys[0]])
    labels=np.array(f[keys[1]])
    names=[el.decode('utf-8')for el in f[keys[2]]]
    f.close()

In [ ]:
df=pd.DataFrame(labels,columns=['label'])
df['class']=[names[l] for l in labels]
pl.figure(figsize=(8,4))
sn.countplot(y='class',data=df,palette='OrRd',alpha=.5)
ti='label distribution'
pl.title(ti,fontsize=16,color=col)
pl.tight_layout(); pl.show()

In [ ]:
N=labels.shape[0]; n=int(.1*N); shuffle_ids=np.arange(N)
np.random.RandomState(12).shuffle(shuffle_ids)
images=images[shuffle_ids]; labels=labels[shuffle_ids]
num_classes2=len(names); img_size2=images.shape[1]
x_test2,x_valid2,x_train2=images[:n],images[n:2*n],images[2*n:]
y_test2,y_valid2,y_train2=labels[:n],labels[n:2*n],labels[2*n:]
df=pd.DataFrame(
    [[x_train2.shape,x_valid2.shape,x_test2.shape],
     [x_train2.dtype,x_valid2.dtype,x_test2.dtype],
     [y_train2.shape,y_valid2.shape,y_test2.shape],
     [y_train2.dtype,y_valid2.dtype,y_test2.dtype]],
    columns=['train','valid','test'],
    index=['image shape','image type','label shape','label type'])
fig=pl.figure(figsize=(8,4)); n=np.random.randint(1,100)
for i in range(n,n+6):
    ax=fig.add_subplot(2,3,i-n+1,xticks=[],yticks=[])
    ax.set_title(
        names[labels[i]],color=col,
        fontdict={'fontsize':'large'})
    ax.imshow((images[i]))
pl.tight_layout(); pl.show(); display(df)

<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff603b;'>Fast Examples</h1>

In [ ]:
origin_img=load_img(file_path+'picture02.png')
x=tf.keras.applications.vgg19\
.preprocess_input(origin_img*255)
x=tf.image.resize(x,(224,224))
vgg19=tf.keras.applications\
.VGG19(include_top=True,weights='imagenet')
prediction_probabilities=vgg19(x)
predicted_top5=tf.keras.applications.vgg19\
.decode_predictions(prediction_probabilities.numpy())[0]
[print((class_name,prob)) 
 for (number,class_name,prob) in predicted_top5]
tensor2img(origin_img)

In [ ]:
detector=th.load(hub_path2).signatures['default']
result=detector(origin_img)
df=pd.DataFrame.from_dict(
    {key:list(value.numpy()) for key,value in result.items()})
df.head(10)

In [ ]:
origin='06_001.png'; style='02_014.png'
def origin2style(origin,style,hub_path):
    get_file(origin); get_file(style)
    hub_module=th.load(hub_path)
    origin_img=load_img(origin); style_img=load_img(style)
    stylized_img=hub_module(
        tf.constant(origin_img),tf.constant(style_img))[0]
    display(tensor2img(stylized_img))
origin2style(origin,style,hub_path)

In [ ]:
origin='01_016.png'; style='02_018.png'
origin2style(origin,style,hub_path)

<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff603b;'>Simple Models </h1>

In [ ]:
def cb(model_weights):
    early_stopping=tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',patience=20,verbose=2)
    checkpointer=tf.keras.callbacks.ModelCheckpoint(
filepath=model_weights,verbose=int(2),save_weights_only=True,
    monitor='val_accuracy',mode='max',save_best_only=True)
    lr_reduction=tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',verbose=2,patience=7,factor=.8)
    return [checkpointer,early_stopping,lr_reduction]

<h2 style='font-family:Aladin; color:#ff603b;'>MLP</h2>

In [ ]:
def mlp_model(img_size,num_classes):
    model=tf.keras.models.Sequential([
        tkl.Flatten(input_shape=(img_size,img_size,3)),
        tkl.Dense(128),tkl.LeakyReLU(alpha=.02),
        tkl.BatchNormalization(),    
        tkl.Dense(256),tkl.LeakyReLU(alpha=.02),
        tkl.BatchNormalization(),    
        tkl.Dense(512),tkl.LeakyReLU(alpha=.02),
        tkl.BatchNormalization(),   
        tkl.Dense(1024),tkl.LeakyReLU(alpha=.02),
        tkl.Dropout(.2),
        tkl.Dense(num_classes,activation='softmax')])
    model.compile(optimizer='nadam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
model=mlp_model(img_size1,num_classes1)
model.fit(x_train1,y_train1,epochs=50,batch_size=16,
          validation_data=(x_valid1,y_valid1),
          callbacks=cb(model_weights));

In [ ]:
model.load_weights(model_weights)
model.evaluate(x_test1,y_test1,verbose=0)

In [ ]:
model=mlp_model(img_size2,num_classes2)
model.fit(x_train2,y_train2,epochs=50,batch_size=64,
          validation_data=(x_valid2,y_valid2),
          callbacks=cb(model_weights))

In [ ]:
model.load_weights(model_weights)
model.evaluate(x_test2,y_test2,verbose=0)

<h2 style='font-family:Aladin; color:#ff603b;'>CNN</h2>

In [ ]:
def cnn_model(img_size,num_classes):
    model=tf.keras.models.Sequential([
        tkl.Conv2D(32,(5,5),padding='same',
                   input_shape=(img_size,img_size,3)),
        tkl.LeakyReLU(alpha=.02),
        tkl.MaxPooling2D(pool_size=(2,2)),
        tkl.Dropout(.25),
        tkl.Conv2D(196,(5,5)),
        tkl.LeakyReLU(alpha=.02),    
        tkl.MaxPooling2D(pool_size=(2,2)),
        tkl.Dropout(.25),
        tkl.GlobalMaxPooling2D(),    
        tkl.Dense(512),tkl.LeakyReLU(alpha=.02),
        tkl.Dropout(.25),
        tkl.Dense(128),tkl.LeakyReLU(alpha=.02),
        tkl.Dropout(.25),
        tkl.Dense(num_classes,activation='softmax')])
    model.compile(optimizer='nadam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
model=cnn_model(img_size1,num_classes1)
model.fit(x_train1,y_train1,epochs=150,batch_size=64,
          validation_data=(x_valid1,y_valid1),
          callbacks=cb(model_weights))

In [ ]:
model.load_weights(model_weights)
model.evaluate(x_test1,y_test1,verbose=0)

In [ ]:
model=cnn_model(img_size2,num_classes2)
model.fit(x_train2,y_train2,epochs=100,batch_size=64,
          validation_data=(x_valid2,y_valid2),
          callbacks=cb(model_weights))

In [ ]:
model.load_weights(model_weights)
model.evaluate(x_test2,y_test2,verbose=0)

<h2 style='font-family:Aladin; color:#ff603b;'>RNN</h2>

In [ ]:
def rnn_model(img_size,hidden,num_classes):
    model=tf.keras.models.Sequential([
        tkl.BatchNormalization(input_shape=(1,3*img_size**2)),
        tkl.LSTM(hidden,return_sequences=True),
        tkl.LSTM(hidden,return_sequences=True),
        tkl.LSTM(hidden),#tkl.Reshape((1,hidden)),
#        tkl.GlobalMaxPooling1D(),
#        tkl.Dense(512),tkl.LeakyReLU(alpha=.02),
#        tkl.Dropout(.5),
        tkl.Dense(num_classes,activation='softmax')])
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='nadam',metrics=['accuracy'])    
    return model

In [ ]:
hidden1=196
model=rnn_model(img_size1,hidden1,num_classes1)
model.fit(
    x_train1.reshape(-1,1,3*img_size1**2),y_train1,
    epochs=50,batch_size=128,callbacks=cb(model_weights),
    validation_data=(x_valid1.reshape(-1,1,3*img_size1**2),
                     y_valid1))

In [ ]:
model.load_weights(model_weights)
model.evaluate(
    x_test1.reshape(-1,1,3*img_size1**2),y_test1)

In [ ]:
hidden2=256
model=rnn_model(img_size2,hidden2,num_classes2)
model.fit(
    x_train2.reshape(-1,1,3*img_size2**2),y_train2,
    epochs=50,batch_size=64,callbacks=cb(model_weights),
    validation_data=(x_valid2.reshape(-1,1,3*img_size2**2),
                     y_valid2))

In [ ]:
model.load_weights(model_weights)
model.evaluate(
    x_test2.reshape(-1,1,3*img_size2**2),y_test2)